## import the librarys

the accuracy is around 70%

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import keras,os
from keras.layers import Conv2D, MaxPool2D,Dropout,Flatten,Dense
from keras.preprocessing import image 
from keras.models import Sequential
from keras.applications.resnet import ResNet50


##import the data

In [ ]:
train_data_gen = image.ImageDataGenerator(rescale= 1./255)
train = train_data_gen.flow_from_directory(directory="/content/drive/MyDrive/x-ray data/train" , target_size=(256,256) , batch_size=32,class_mode= 'binary')

Found 4852 images belonging to 2 classes.


In [ ]:
train.class_indices

{'COVID': 0, 'Normal': 1}

In [ ]:
train_data_gen = image.ImageDataGenerator(rescale= 1./255)
test = train_data_gen.flow_from_directory(directory="/content/drive/MyDrive/x-ray data/test" , target_size=(256,256) , batch_size=32,class_mode= 'binary')

Found 240 images belonging to 2 classes.


In [ ]:
test.class_indices

{'COVID': 0, 'Normal': 1}

##resnet_x( Transfer learning)

In [ ]:
resnet_model = Sequential()

pretrained_model= ResNet50(include_top=False,
                   input_shape=(256,256,3),
                   pooling='avg',
                   weights='imagenet')
for layer in pretrained_model.layers:
        layer.trainable=False

resnet_model.add(pretrained_model)

94781440/94765736 [==============================] - 1s 0us/step


In [ ]:
resnet_model.add(Flatten())
resnet_model.add(Dense(512, activation='relu'))
resnet_model.add(Dense(1, activation='sigmoid'))

In [ ]:

resnet_model.compile(loss=keras.losses.binary_crossentropy, optimizer="adam",metrics=['acc'])

##check the model

In [ ]:
resnet_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet50 (Functional)       (None, 2048)              23587712  
                                                                 
 flatten (Flatten)           (None, 2048)              0         
                                                                 
 dense (Dense)               (None, 512)               1049088   
                                                                 
 dense_1 (Dense)             (None, 1)                 513       
                                                                 
Total params: 24,637,313
Trainable params: 1,049,601
Non-trainable params: 23,587,712
_________________________________________________________________


##train the data

In [ ]:
resnet_model.fit_generator(train, steps_per_epoch=10,epochs=10,validation_steps=8, validation_data=test)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  """Entry point for launching an IPython kernel.


Epoch 1/10
10/10 [==============================] - 174s 16s/step - loss: 0.9942 - acc: 0.5063 - val_loss: 1.1469 - val_acc: 0.5000
Epoch 2/10
10/10 [==============================] - 115s 12s/step - loss: 0.7902 - acc: 0.5031 - val_loss: 0.7852 - val_acc: 0.5000
Epoch 3/10
10/10 [==============================] - 115s 12s/step - loss: 0.6722 - acc: 0.6344 - val_loss: 0.7122 - val_acc: 0.5000
Epoch 4/10
10/10 [==============================] - 115s 11s/step - loss: 0.6269 - acc: 0.6656 - val_loss: 0.7621 - val_acc: 0.5000
Epoch 5/10
10/10 [==============================] - 114s 11s/step - loss: 0.6828 - acc: 0.5312 - val_loss: 0.6517 - val_acc: 0.5917
Epoch 6/10
10/10 [==============================] - 114s 11s/step - loss: 0.6338 - acc: 0.6656 - val_loss: 0.6289 - val_acc: 0.6708
Epoch 7/10
10/10 [==============================] - 112s 11s/step - loss: 0.6470 - acc: 0.6156 - val_loss: 0.6619 - val_acc: 0.5583
Epoch 8/10
10/10 [==============================] - 112s 11s/step - loss: 0.

In [ ]:
# list all data in history
#print(history.history.keys())
keras.callbacks.History()

In [ ]:
#history = resnet_model.fit(np.arange(100).reshape(5, 20), np.zeros(5),epochs=10)
print(history.params)

NameError: ignored

In [ ]:
loss_train = history.history['acc']
loss_val = history.history['val_acc']
epochs = range(1,11)
plt.plot(epochs, loss_train, 'g', label='Training accuracy')
plt.plot(epochs, loss_val, 'b', label='validation accuracy')
plt.title('Training and Validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

NameError: ignored

## random image

non covid random image

In [ ]:
path= "/content/normal.jpeg"
img=image.load_img(path,target_size=(256,256))
img= image.img_to_array(img)/255
img=np.array([img])
img.shape

(1, 256, 256, 3)

In [ ]:
resnet_model.predict(img)

array([[0.4165893]], dtype=float32)

covid image

In [ ]:
path= "/content/pneumonia.jpeg"
img=image.load_img(path,target_size=(256,256))
img= image.img_to_array(img)/255
img=np.array([img])
img.shape

(1, 256, 256, 3)

In [ ]:
resnet_model.predict(img)

array([[0.36021176]], dtype=float32)